# Using Bounds to Fail to QPP

Demonstrating how bounds can be employed to cause VMCON to fail. In my opinion, this is *likely* what is causing the QPP to immediately fail in PROCESS.

In [ ]:
X0 = 1.5

def f(x):
    return x**2

def df(x):
    return 2*x


def c(x):
    return x**4 - (3*x**2) - 1

def dc(x):
    return 4*x**3 - (6*x)

In [ ]:
from plotly.subplots import make_subplots
import numpy as np

xs = np.linspace(-2, 2)

fig = make_subplots(rows=2, cols=2, subplot_titles=("Objective function f(x)", "Constraint function c(x)", "", "Derivative constraint function c'(x)"))

fig.add_scatter(
    x=xs, y=f(xs), name="f(x)", row=1, col=1
)

fig.add_scatter(
    x=xs, y=df(xs), name="f'(x)", row=1, col=1
)

fig.add_scatter(
    x=xs, y=c(xs), name="c(x)", row=1, col=2
)

fig.add_scatter(
    x=[xs[0], xs[-1]], 
    y=[0, 0], 
    name="Constraint lower bound", 
    mode="lines",
    line={"dash": "dash"},
    row=1, 
    col=2,
)

fig.add_scatter(
    x=xs, y=dc(xs), name="c'(x)", row=2, col=2
)

fig.add_vline(x=X0, line_dash="dash")

fig.show()


In [ ]:
print(f"f'(x0) = {df(X0)}")
print(f"c'(x0) = {dc(X0)}")

## QPP

Because this problem is 1D, we can simplify the quadratic program that VMCON attempts to solve.

The QPP returns the **search direction, $\delta$,** for a feasible optima. In the 1D case:
* $\delta > 0$ means we search to the 'right' (increasing `x` direction) for a feasible optima.
* $\delta < 0$ means we search to the 'left' (decreasing `x` direction) for a feasible optima.

In [ ]:
import plotly.graph_objects as go

def Q_for_x0(delta):
    # this is reduced because we have a 1D function and no hessian because 1D identity is 1
    return f(X0) + delta*df(X0) + 0.5*delta**2

def constraint_for_x0(delta):
    return dc(X0)*delta + c(X0)

deltas = np.linspace(-2, 2)

fig = go.Figure()

fig.add_scatter(x=deltas, y=Q_for_x0(deltas), name="Q(delta) | x0")
fig.add_scatter(x=deltas, y=constraint_for_x0(deltas), name="QPP constraint | x0")
fig.add_scatter(
    x=[deltas[0], deltas[-1]], 
    y=[0, 0], 
    name="QPP constraint lower bound",
    mode="lines",
    line={"dash": "dash"}
)
fig.update_layout(xaxis_title="delta")

In [ ]:
delta_bound = (- c(X0))/dc(X0)

print(
    f"QPP will be solvable for delta {'<=' if dc(X0) < 0 else '>='} " 
    f"{delta_bound}"
)

We can verify the above by running VMCON and should expect $\delta_1$ to be equal to the value above.

In [ ]:
from pyvmcon import Problem

problem = Problem(
    f=f,
    df=df,
    equality_constraints=[],
    inequality_constraints=[lambda x: c(x)],
    dequality_constraints=[],
    dinequality_constraints=[lambda x: dc(x)]
)

In [ ]:
from pyvmcon import solve

x, _, _, result = solve(
    problem, 
    np.array([X0]), 
    callback=lambda i, _r, x, _cp: print(f"delta{i} = {x - X0}"),
)

## Bounding `x`

We can setup a bound such that the QPP becomes unsolvable because the linearised constraint and bound specify disjoint bounds on the delta.

In [ ]:
x, _, _, result = solve(
    problem, 
    np.array([X0]), 
    ubs=np.array([1.6]),
    callback=lambda i, _r, x, _cp: print(f"x({i}) = {x}"),
)